梯度下降demo 第一个是给定权重，第二个是加载csv，随机正态分布权重

In [5]:
import numpy as np

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1/(1+np.exp(-x))

def sigmoid_prime(x):
    """
    # Derivative of the sigmoid function
    """
    return sigmoid(x) * (1 - sigmoid(x))

learnrate = 0.5
x = np.array([1, 2, 3, 4])
y = np.array(0.5)

# Initial weights
w = np.array([0.5, -0.5, 0.3, 0.1])

### Calculate one gradient descent step for each weight
### Note: Some steps have been consilated, so there are
###       fewer variable names than in the above sample code

# TODO: Calculate the node's linear combination of inputs and weights
h = np.dot(x,w)
print("输入和权重的线性组合",h)

# TODO: Calculate output of neural network
nn_output = sigmoid(h)
print("神经网络输出 yhat",nn_output)

# TODO: Calculate error of neural network
error = y-nn_output
print("输出误差 error (y - y-hat)", error)
# TODO: Calculate the error term
#       Remember, this requires the output gradient, which we haven't
#       specifically added a variable for.

# output gradient (f'(h))
# 输出梯度
output_grad = sigmoid_prime(h)
print("输出梯度 output gradient (f'(h))", output_grad)

# error term (lowercase delta)
error_term = error * sigmoid_prime(h)
print("error term (lowercase delta)",error_term)


# TODO: Calculate change in weights
print("learnrate * error_term =",learnrate * error_term)
del_w = learnrate * error_term * x
print("梯度下降一步", del_w, x, w)

print('Neural Network output:')
print(nn_output)
print('Amount of Error:')
print(error)
print('Change in Weights:')
print(del_w)

输入和权重的线性组合 0.7999999999999998
神经网络输出 yhat 0.6899744811276125
输出误差 error (y - y-hat) -0.1899744811276125
输出梯度 output gradient (f'(h)) 0.2139096965202944
error term (lowercase delta) -0.04063738360460799
learnrate * error_term = -0.020318691802303994
梯度下降一步 [-0.02031869 -0.04063738 -0.06095608 -0.08127477] [1 2 3 4] [ 0.5 -0.5  0.3  0.1]
Neural Network output:
0.6899744811276125
Amount of Error:
-0.1899744811276125
Change in Weights:
[-0.02031869 -0.04063738 -0.06095608 -0.08127477]


In [ ]:
课程3 的代码：

In [ ]:
准备数据


In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#花点
def plot_points(X, y):
    admitted = X[np.argwhere(y==1)]
    rejected = X[np.argwhere(y==0)]
    plt.scatter([s[0][0] for s in rejected], [s[0][1] for s in rejected], s = 25, color = 'blue', edgecolor = 'k')
    plt.scatter([s[0][0] for s in admitted], [s[0][1] for s in admitted], s = 25, color = 'red', edgecolor = 'k')


    
    
    
admissions = pd.read_csv('student_data.csv')
# Make dummy variables for rank
#one-hot 编码，将rank 分类
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features 
#标准化
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(),data[field].std()
    data.loc[:,field] = (data[field] - mean) / std
    

# Split off random 10% of the data for testing
#10%数据用来测试
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.loc[sample], data.drop(sample)

# Split into features and targets
features,targets=data.drop('admit', axis=1), data['admit']
features_test,targets_test = test_data.drop('admit', axis=1), test_data['admit']



In [21]:
def sigmoid(x):
    return 1/(1+np.exp(-x))


np.random.seed(42)
n_records, n_features = features.shape
last_loss = None
# Initialize weights
# 初始化权重
weights = np.random.normal(scale=1/n_features**.5, size=n_features)
print("初始权重：", weights)
# Neural Network hyperparameters
# 神经网络超参数
epochs = 1000
learnrate = 0.1

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x,y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target
        # 遍历所有记录，x是输入，y是目标，
        output= sigmoid(np.dot(weights, x))
        
        #计算误差
        error = y - output
        # TODO: Calculate the error term (lowercase delta)
        # yhat*(1-yhat) 梯度 
        error_term = error*output*(1-output)
        
        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term *x
        
    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w / n_records
    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        print(e, weights)
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets)**2)
        if last_loss and last_loss < loss:
            print("训练损失：", loss, " 损失在增大！")
        else:
          print("训练损失：", loss)
        last_loss = loss

        
#计算测试数据的准确性
test_out = sigmoid(np.dot(features_test,  weights))
predictions = test_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("预测准确性:{:.3f}".format(accuracy))
        

初始权重： [ 0.2027827  -0.05644616  0.26441774  0.62177434 -0.09559271 -0.09558601]
0 [ 0.20357284 -0.05453858  0.26431397  0.61937445 -0.09724029 -0.0968585 ]
训练损失： 0.2638998873467431
100 [ 0.24421438  0.08752405  0.25274432  0.39493348 -0.24567134 -0.21507531]
训练损失： 0.24201811706739843
200 [ 0.24379311  0.16227971  0.24062355  0.20599237 -0.36778107 -0.31808849]
训练损失： 0.22856131388511527
300 [ 0.23257     0.20167518  0.22899584  0.05178606 -0.46991038 -0.4090788 ]
训练损失： 0.2196744183204822
400 [ 0.22194525  0.22336476  0.21813352 -0.07232641 -0.55598824 -0.49010051]
训练损失： 0.21364426361654243
500 [ 0.21490658  0.23628796  0.2080968  -0.17170469 -0.62889384 -0.56261808]
训练损失： 0.20948211975991676
600 [ 0.21137508  0.24479869  0.19886929 -0.25129992 -0.69094559 -0.62779579]
训练损失： 0.20655596596517978
700 [ 0.21046528  0.25098911  0.1904049  -0.31526607 -0.74404219 -0.68660661]
训练损失： 0.20445656190984315
800 [ 0.21128645  0.25587337  0.1826479  -0.36692397 -0.78972948 -0.73987611]
训练损失： 0.202918

In [17]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.1

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output
        output = sigmoid(np.dot(weights, x))

        # TODO: Calculate the error
        error = y - output

        # TODO: Calculate the error term
        error_term = error*output*(1-output)

        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w +=  error_term * x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.2638998873467431
Train loss:  0.24201811706739843
Train loss:  0.22856131388511527
Train loss:  0.2196744183204822
Train loss:  0.21364426361654243
Train loss:  0.20948211975991676
Train loss:  0.20655596596517978
Train loss:  0.20445656190984315
Train loss:  0.20291884350403244
Train loss:  0.20176991624983903
Prediction accuracy: 0.750
